In [ ]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import scipy

# Load and prepare the graph
def load_graph(file_path):
    G = nx.DiGraph()
    with open(file_path, "r") as file:
        for line in file:
            if not line.startswith("#"):
                from_node, to_node = map(int, line.strip().split())
                G.add_edge(from_node, to_node)
    return G

# Convert to undirected graph for visualization
def convert_to_undirected(nx_graph):
    return nx_graph.to_undirected()

# Visualize a subgraph
def visualize_subgraph(G, num_nodes=200):
    subgraph_nodes = list(G.nodes)[:num_nodes]
    subgraph = G.subgraph(subgraph_nodes)
    plt.figure(figsize=(12, 8))
    pos = nx.spring_layout(subgraph)
    nx.draw(subgraph, pos, with_labels=True, node_size=100, font_size=3)
    plt.title("Subgraph Visualization (First {} Nodes)".format(num_nodes))
    plt.show()
    return subgraph

# Bron-Kerbosch algorithm for finding cliques
def bron_kerbosch(graph, r, p, x, cliques):
    if not p and not x:
        cliques.append(r)
        return

    pivot = max((p | x), key=lambda u: len(graph[u] & p)) if p or x else None
    for v in p - (graph[pivot] if pivot else set()):
        neighbors = graph[v]
        bron_kerbosch(graph, r | {v}, p & neighbors, x & neighbors, cliques)
        p.remove(v)
        x.add(v)

# Convert to undirected adjacency for Bron-Kerbosch
def convert_to_undirected_adjacency(nx_graph):
    undirected_graph = nx_graph.to_undirected()
    adj = {node: set(undirected_graph.neighbors(node)) for node in undirected_graph.nodes()}
    return adj

# Visualize the largest clique
def visualize_largest_clique(subgraph, largest_clique):
    plt.figure(figsize=(12, 8))
    pos = nx.spring_layout(subgraph)
    nx.draw(subgraph, pos, with_labels=True, node_size=100, font_size=3)
    nx.draw_networkx_nodes(subgraph, pos, nodelist=list(largest_clique), node_color='r')
    plt.title("Visualization Highlighting the Largest Maximal Clique")
    plt.show()

# Enhanced visualization for cliques with the largest clique highlighted
def visualize_cliques_with_rainbow(subgraph, found_cliques, largest_clique):
    plt.style.use('dark_background')
    plt.figure(figsize=(20, 20), dpi=300, facecolor='black')
    pos = nx.spring_layout(subgraph, k=1.5, iterations=100)
    nx.draw_networkx_edges(subgraph, pos, edge_color='white', alpha=0.15, width=2)
    
    node_colors = {}
    for idx, clique in enumerate(found_cliques):
        color = plt.cm.rainbow(idx / len(found_cliques))
        for node in clique:
            if node not in node_colors:
                node_colors[node] = color

    for node in largest_clique:
        node_colors[node] = 'red'  # Highlight the largest clique with a bright color
    
    node_color_list = [node_colors.get(node, (0.5, 0.5, 0.5, 0.3)) for node in subgraph.nodes()]
    node_size_list = [600 if node in largest_clique else 150 for node in subgraph.nodes()]  # Make largest clique nodes larger
    nx.draw_networkx_nodes(subgraph, pos, node_size=node_size_list, node_color=node_color_list, alpha=0.8)
    nx.draw_networkx_labels(subgraph, pos, font_size=6, font_color='white', font_weight='light')

    # Add a legend
    legend_elements = [
        plt.Line2D([0], [0], marker='o', color='w', label='Largest Clique', markersize=10, markerfacecolor='red'),
        plt.Line2D([0], [0], marker='o', color='w', label='Other Cliques', markersize=10, markerfacecolor='blue'),
    ]
    plt.legend(handles=legend_elements, loc='upper right', fontsize=8)

    plt.title("All Maximal Cliques with Rainbow Coloring and Largest Clique Highlighted", color='white', pad=20, size=14)
    plt.axis('off')
    plt.show()

# Generate a summary of the graph
def graph_summary(G, largest_clique):
    print("Graph Summary:")
    print(f"- Number of nodes: {G.number_of_nodes()}")
    print(f"- Number of edges: {G.number_of_edges()}")
    print(f"- Largest clique size: {len(largest_clique)}")
    print(f"- Nodes in the largest clique: {sorted(largest_clique)}")

# Main Execution
file_path = "Data/Wiki-Vote.txt"
G = load_graph(file_path)
print(f"Graph has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")

# Convert the graph to undirected for visualization
G_undirected = convert_to_undirected(G)

subgraph = visualize_subgraph(G_undirected, num_nodes=200)
adj_graph = convert_to_undirected_adjacency(subgraph)

found_cliques = []
bron_kerbosch(adj_graph, set(), set(adj_graph.keys()), set(), found_cliques)

largest_clique = max(found_cliques, key=len)

# Generate summary
graph_summary(subgraph, largest_clique)

# Visualize
visualize_largest_clique(subgraph, largest_clique)
visualize_cliques_with_rainbow(subgraph, found_cliques, largest_clique)
